In [1]:
!pip install tetris_gymnasium


In [2]:
import numpy as np
import random
import gymnasium as gym
import tetris_gymnasium
from tetris_gymnasium.envs.tetris import ActionsMapping

In [3]:
#Hyperparameters:
KVAL = 7
PIECE_WEIGHT = 20

In [4]:
def printarray(array):
    for row in array:
        print(row)

In [5]:
board1 = [[0, 0, 0, 0, 0, 0, 8, 0, 0, 0],
        [0, 0, 0, 0, 8, 8, 8, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]


#Finds the column heights for attached pieces
def find_column_heights(board):
    column_heights = [0] * 10
    for c in range(10):
        maxHeight = 0
        r = 19
        while r >= 0:
            if board[r][c] != 0 and is_attached(board, c, r):
                maxHeight = 20 - r
            r-=1
        column_heights[c] = maxHeight
    return column_heights



def is_valid(board, xindex, yindex):
    #Checking out of bounds
    if(yindex >= 20 or yindex < 0 or xindex < 0 or xindex >= 10):
        return False
    #Check if space is not a piece
    elif(board[yindex][xindex] == 0):
        return False
    else:
        return True


#Tested and Approved
#Checks if the piece which contains the position xindex yindex is connected
def is_attached(board, xindex, yindex):
    if(not is_valid(board, xindex, yindex)):
        return False
    queue = [(xindex, yindex)]
    visited = []
    while(len(queue) > 0):
        current_index = queue.pop()
        #print(current_index)
        visited.append(current_index)
        x = current_index[0]
        y = current_index[1]
        if y == 19:
            return True
        if(is_valid(board, x, y-1) and not ((x, y-1) in visited)):
            queue.append((x, y-1))
        if(is_valid(board, x+1, y) and not ((x+1, y) in visited)):
            queue.append((x+1, y))
        if(is_valid(board, x-1, y) and not ((x-1, y) in visited)):
            queue.append((x-1, y))
        if(is_valid(board, x, y+1) and not ((x, y+1) in visited)):
            queue.append((x, y+1))

    return False


#Used to find the coordinates of the active piece
def get_visited(board, xindex, yindex):
    #print(board[yindex][xindex])
    #printarray(board)
    #print(board[yindex])
    if(not is_valid(board, xindex, yindex)):
        #Stands for No Piece Found
        return "NPF"
    queue = [(xindex, yindex)]
    visited = []
    while(len(queue) > 0):
        current_index = queue.pop()
        #print(current_index)
        visited.append(current_index)
        x = current_index[0]
        y = current_index[1]
        if y == 19:
            return visited
        if(is_valid(board, x, y-1) and not ((x, y-1) in visited)):
            queue.append((x, y-1))
        if(is_valid(board, x+1, y) and not ((x+1, y) in visited)):
            queue.append((x+1, y))
        if(is_valid(board, x-1, y) and not ((x-1, y) in visited)):
            queue.append((x-1, y))
        if(is_valid(board, x, y+1) and not ((x, y+1) in visited)):
            queue.append((x, y+1))

    return visited

#is_attached_with_visited(board1, 5,1)



In [6]:
#Finds valid unattached pieces
def is_unattached(board, x_index, y_index):
    if(board[y_index][x_index] == 0):
        #print(f"There is no piece at {x_index, y_index}")
        return False
    elif(not is_attached(board, x_index, y_index)):
        return True
    return False

def find_active_piece(board):
    piece = 0
    xindex = 0
    yindex = 0
    #Row by row scan starting at the top to find the first piece which is unattached
    while(not is_unattached(board, xindex, yindex)):
        if(xindex == 9):
            xindex = 0
            yindex += 1
        else:
            xindex += 1
        if(yindex >= 20):
            #We do this so the is_unattached method doesn't have to deal with out of bounds
            break
    if(yindex < 20):
        piece = board[yindex][xindex]
    if(piece == 0):
        pass
        #print("No active piece found")
    piece_coordinates = get_visited(board, xindex, yindex)
    return piece_coordinates

def find_piece_width(board):
    piece_coordinates = find_active_piece(board)
    x_coordinates = []
    for coordinate in piece_coordinates:
        x_coordinates.append(coordinate[0])
    if(x_coordinates == ["N", "P", "F"]):
        return "No Piece Found"
    #print(x_coordinates)
    #print(x_coordinates == ["N", "P", "F"])
    range = max(x_coordinates) - min(x_coordinates) + 1
    return range

find_piece_width(board1)

def find_leftmost_index(board):
    piece_coordinates = find_active_piece(board)
    x_coordinates = []
    for coordinate in piece_coordinates:
        x_coordinates.append(coordinate[0])
    return min(x_coordinates)



In [7]:
#Finds the location where the robot should aim
def find_aim(column_heights, piece_width):
    if(piece_width == 1):
        aim = np.argmin(column_heights)
    elif(piece_width == 2):
        lowest_2by = []
        #Find the maxes of the current spot and the spot next to it to find the lowest point to fit a 2 wide piece
        for i in range(len(column_heights)-1):
            lowest_2by.append(max(column_heights[i],column_heights[i+1]))
        aim = np.argmin(lowest_2by)
    else:
        print("Piece width exceeded 2, should have been rotated")
    return aim




In [8]:

# --- Initialize the Tetris Environment ---
env = gym.make("tetris_gymnasium/Tetris", render_mode="ansi")

# Get the available actions from ActionsMapping
actions = ActionsMapping()

#Convert string board which is returned by env.render() to an array
def convert_strBoard_to_2dArray(board):
    array = []
    row = []
    for char in board:
        if char != "\n":
            if char == '.':
                row.append(0)
            else:
                row.append(int(char))
        else:
            array.append(row)
            row = []
    array.append(row)
    #printarray(array)
    return array

# --- Function to Extract Board Features ---
def get_board_features(board):
    """
    Extracts meaningful features from the Tetris board.
    """
    column_heights = [0] * 10
    for c in range(10):
        maxHeight = 0
        r = 19
        while r >= 0:
            if board[r][c] != 0:
                maxHeight = 20 - r
            r-=1
        column_heights[c] = maxHeight
    #print(column_heights)
    #printarray(board)
    max_height = max(column_heights)
    bumpiness = sum(abs(column_heights[i] - column_heights[i+1]) for i in range(9))
    holes = sum(1 for c in range(10) for r in range(column_heights[c]) if board[r][c] == 0)
    return np.array([max_height, holes, bumpiness])


def play_tetris():
    """
    Plays a single game of Tetris using a genome's weights.
    """
    total_lines_cleared = 0
    total_reward = 0
    state = env.reset()
    done = False

    move_list = []

    while not done:
        board = env.render()  # Get board as image

        #print(type(board))
        board = convert_strBoard_to_2dArray(board)
        column_heights = find_column_heights(board)
        #print(f"column heights: {column_heights}")
        piece_width = find_piece_width(board)
        if(piece_width == "No Piece Found"):
            #Hard drop if the active piece doesn't have anywhere to go
            action = 5
        #Rotate the piece if it is wide
        elif(piece_width > 2):
            action = 3 #Rotate clockwise
        else:
            aim = find_aim(column_heights, piece_width)
            left_index = find_leftmost_index(board)
            if(left_index > aim):
                #Move left
                action = 0
            elif(left_index < aim):
                #Move right
                action = 1
            else:
                #We have found the columns we want the piece to go
                #Hard drop
                action = 5
        #print(action)
        #print(f"action: {action}")
        # Ensure the action is valid
        #action = actions.get(action)  # Map to an action from ActionsMapping
        step_result = env.step(action)  # Store step results in a variable

        # Handle both 4-value and 5-value return cases
        if len(step_result) == 4:
            next_state, reward, done, info = step_result  # Old format
        else:
            next_state, reward, done, truncated, info = step_result  # New format
            done = done or truncated  # Ensure 'done' includes truncation
        #print(truncated)
        total_reward += reward
        #print(info['lines_cleared'])
        #if(info['lines_cleared'] > 0):
            #printarray(board)
            #print(next_state)
        piece_coordinates = find_active_piece(board)
        #Format the piece coordinates
        if(piece_coordinates == "NPF" or len(piece_coordinates) != 4):
            piece_coordinates = [(-1, -1), (-1, -1), (-1, -1), (-1, -1)]
        total_lines_cleared += info['lines_cleared']  # Track lines cleared

        move = {"board": board, "piece": piece_coordinates, "action": action, "reward": reward}
        move_list.append(move)

    #print(total_lines_cleared)
    return {"Lines Cleared" : total_lines_cleared, "Reward": total_reward, "Move List":move_list}

def train_model(episodes):
    history = []
    i = 0
    while(i < episodes):
        tetris = play_tetris()
        reward = tetris["Reward"]
        lines_cleared = tetris["Lines Cleared"]
        for move_list in tetris["Move List"]:
            if(len(move_list["piece"]) < 4):
                print(move_list["piece"])
        history.append(tetris)
        print(f"Episode {i}: Reward: {reward}, Lines Cleared: {lines_cleared}")
        i += 1
    return history



In [9]:
history = train_model(20)

Episode 0: Reward: 27, Lines Cleared: 0
Episode 1: Reward: 26, Lines Cleared: 0
Episode 2: Reward: 55, Lines Cleared: 2
Episode 3: Reward: 26, Lines Cleared: 0
Episode 4: Reward: 26, Lines Cleared: 0
Episode 5: Reward: 30, Lines Cleared: 0
Episode 6: Reward: 41, Lines Cleared: 1
Episode 7: Reward: 26, Lines Cleared: 0
Episode 8: Reward: 24, Lines Cleared: 0
Episode 9: Reward: 27, Lines Cleared: 0
Episode 10: Reward: 29, Lines Cleared: 0
Episode 11: Reward: 52, Lines Cleared: 2
Episode 12: Reward: 41, Lines Cleared: 1
Episode 13: Reward: 26, Lines Cleared: 0
Episode 14: Reward: 53, Lines Cleared: 2
Episode 15: Reward: 37, Lines Cleared: 1
Episode 16: Reward: 28, Lines Cleared: 0
Episode 17: Reward: 40, Lines Cleared: 1
Episode 18: Reward: 39, Lines Cleared: 1
Episode 19: Reward: 41, Lines Cleared: 1


In [10]:
sum = 0
count = 0
for episode in history:
    sum += episode["Lines Cleared"]
    count += 1
print(sum/count)

0.6


In [11]:
def extract_features(history):
    data = []
    for episode in history:
        move_list = episode["Move List"]
        for move in move_list:
            move_data = []
            board = move["board"]
            piece = move["piece"]
            action = move["action"]
            #add each
            for row in board:
                for element in row:
                    move_data.append(element)
            #move_data.append("|") #Used for debugging, remove later
            for pixel in piece:
                for coordinate in pixel:
                    #print(coordinate)
                    move_data.append(coordinate)
            move_data.append(action)
            if(len(move_data) != 209):
                print(move_data)
                print()
            data.append(move_data)
    return data
features_and_label = extract_features(history)
print(features_and_label[len(features_and_label)-1])
print(len(features_and_label))

[0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 7, 0, 0, 0, 0, 0, 2, 0, 0, 0, 7, 0, 0, 0, 6, 0, 8, 8, 5, 7, 7, 4, 0, 6, 6, 0, 0, 8, 5, 5, 4, 4, 7, 6, 0, 2, 0, 8, 0, 5, 4, 4, 7, 3, 3, 2, 0, 8, 8, 4, 4, 7, 7, 3, 3, 2, 0, 0, 8, 0, 4, 5, 0, 0, 7, 2, 6, 0, 8, 3, 3, 5, 5, 0, 7, 6, 6, 2, 7, 3, 3, 0, 5, 7, 7, 6, 4, 2, 7, 0, 6, 0, 4, 3, 3, 0, 4, 2, 5, 6, 8, 8, 4, 3, 3, 8, 8, 2, 5, 5, 0, 8, 0, 3, 3, 0, 8, 2, 0, 5, 0, 8, 2, 0, 4, 5, 8, 2, 0, 7, 0, 6, 2, 4, 4, 5, 5, 2, 0, 7, 6, 6, 2, 0, 4, 0, 5, 2, 7, 7, 6, 0, 2, -1, -1, -1, -1, -1, -1, -1, -1, 0]
2414


In [12]:

def extract_current_features(board, piece):
    move_data = []
    for row in board:
        for element in row:
            move_data.append(element)
            #print(piece[0][0])
    for pixel in piece:
        for coordinate in pixel:
            #print(coordinate)
            move_data.append(coordinate)
    if(len(move_data) > 209):
        move_data = move_data[0:208]
    return move_data



In [13]:

def find_distance(list1, list2, piece_weight):
    sum = 0
    piece1 = list1[len(list1)-8:]
    piece2 = list2[len(list2)-8:]
    #print(piece1)
    #Extract the coordinates again
    c11 = (piece1[0], piece1[1])
    c12 = (piece1[2], piece1[3])
    c13 = (piece1[4], piece1[5])
    c14 = (piece1[6], piece1[7])
    c21 = (piece2[0], piece2[1])
    c22 = (piece2[2], piece2[3])
    c23 = (piece2[4], piece2[5])
    c24 = (piece2[6], piece2[7])

    piece1_coordinates = set([c11, c12, c13, c14])
    piece2_coordinates = set([c21, c22, c23, c24])

    """for e in piece1_coordinates:
        print(f"1{e}")
    for e in piece2_coordinates:
        print(f"2{e}")"""

    set_union = list(piece1_coordinates & piece2_coordinates)
    #print(f"Set union: {set_union}")

    for i in range(len(list1)):
        try:
            if i >= len(list1)-8:
            #This only happens if the feature is the piece
                #we will add the intersection of the pieces at the end
                pass
            else:
                pos1 = list1[i]
                pos2 = list2[i]
                if pos1 > 0:
                    pos1 = 0
                if pos2 > 0:
                    pos2 = 0
                sum += np.abs(pos1 - pos2)
        except:
            pass
    sum += len(set_union) * piece_weight
    return sum

find_distance([9, 4, 5, 9, 3, 4, 5, 6, 3, 6, 0], [9, "H", 9, 3,5,6,7,4,5,4,2], 10)

10

In [15]:
import statistics as stats

def make_prediction(data, current_board, current_piece, k_val, piece_weight):
    data = np.array(data)
    features = data[:, [0, len(data[0])-2]] #Selecting all rows but only columns from the first until the second to last
    label = data[:, len(data[0])-1]        #Last column (this is the move which is the label)
    features = data
    distances = []
    current_features = extract_current_features(current_board, current_piece)
    i = 0
    for move in features:
        if(len(current_features) >= len(move)):
            print(len(current_features), len(move))
        dist = find_distance(current_features, move, piece_weight)
        distances.append(dist)
        i += 1
    #This will get the index of the minimum distance and take the label at that distance meaning k=1 right now
    smallest_indices = label[np.argsort(distances)[:k_val]]
    return stats.mode(smallest_indices)

make_prediction(features_and_label, board1, find_active_piece(board1), KVAL, PIECE_WEIGHT)


0

In [19]:
def play_game(k_val, piece_weight):
    """
    Plays a single game of Tetris using a genome's weights.
    """
    total_lines_cleared = 0
    total_reward = 0
    state = env.reset()
    done = False

    move_list = []

    while not done:
        board = env.render()  # Get board as image

        #print(type(board))
        board = convert_strBoard_to_2dArray(board)
        #printarray(board)
        #print()
        action = make_prediction(features_and_label, board, find_active_piece(board), k_val, piece_weight)
        #print(action)
        #print(f"action: {action}")
        # Ensure the action is valid
        #action = actions.get(action)  # Map to an action from ActionsMapping
        step_result = env.step(action)  # Store step results in a variable

        # Handle both 4-value and 5-value return cases
        if len(step_result) == 4:
            next_state, reward, done, info = step_result  # Old format
        else:
            next_state, reward, done, truncated, info = step_result  # New format
            done = done or truncated  # Ensure 'done' includes truncation
        total_reward += reward
    return total_reward



19

In [ ]:
#Hyperparameter Tuning
max = 0
maxk = 0
maxw = 0
for k in range(3,9,2):
  for piece_weight in range(5, 25, 5):
    for i in range(1):
      reward = play_game(k, piece_weight)
      if(reward > max):
        max = reward
        maxk = k
        maxw = piece_weight
      print(f"k: {k}, piece_weight: {piece_weight}, reward: {reward}")

k: 3, piece_weight: 5, reward: 14
k: 3, piece_weight: 10, reward: 12
k: 3, piece_weight: 15, reward: 14
k: 3, piece_weight: 20, reward: 15
k: 5, piece_weight: 5, reward: 18
